# Install Packages:

In [ ]:
#first time run this:
#using Pkg; Pkg.add(["Flux","ClimateTools","BSON","GeoMakie", "CairoMakie"])

# Flux Basics

My first MLP:

Flux imports some structures (classes) and methods in your namesapce by default, such as `Dense()` and `σ`.

In [ ]:
using Flux

In [ ]:
model = Chain(
  Dense(2, 3, σ), # 2 input features, 3 output features, sigmoid activation
  Dense(3, 1)
)

Note: you should normally be able to access the docstring via `?Dense`. But there is a vscode bug that returns an error related to Latex...
Here is a workaround:

In [ ]:
import Markdown
Base.showable(::MIME"text/markdown", ::Markdown.MD) = false

In [ ]:
?Dense

Another way to get information about all methods with the same name:

In [ ]:
methods(Dense)

Back to our model: Access weights and biases

In [ ]:
typeof(model[1])

In [ ]:
model[1].weight

In [ ]:
model[1].bias

In [ ]:
x = rand(Float32,2) # input features
model(x) # output

## Loss Functions

In [ ]:
methods(Flux.mse)

In [ ]:
methods(Flux.crossentropy)

# Climate Data Example: Train a CNN on ERA5

We want to train a CNN to classify the month of year.

In [ ]:
using ClimateTools


## check out dataset that comes with the repo:

In [ ]:
Dataset("era5_sst_global_5.nc")

Load data into memory:

In [ ]:
ssts = load("era5_sst_global_5.nc", "sst")

## Visualize data:

In [ ]:
using GeoMakie, CairoMakie
fig = Figure(size = (500, 300), Contour = (; labelsize = 14, labelfont = :bold))
ax1 = GeoAxis(fig[1,1]; title = "ERA5 SSTs at $(ssts.data.axes[3][1])")
contourf!(ax1, ssts.data.axes[1][:], ssts.data.axes[2][:], ssts.data.data[:,:,1])
fig

In [ ]:
size(ssts.data)

In [ ]:
time_dim = size(ssts.data.axes[3])[1]
println("The dataset has $time_dim months of data")

# Define the number of years and months
num_years = Int(time_dim ./ 12) #broadcasting division
num_months = 12

# Subset the final 10 years of data for testing
test_years = 10
train_years = num_years .- test_years;

In [ ]:
train_data = replace(ssts.data.data[:,:,1:(train_years * num_months)], NaN => 280.0)
test_data = replace(ssts.data.data[:,:,(train_years * num_months + 1):end], NaN => 280.0) ; # use ; to suppress output

In [ ]:
ys = 1:12
ys_train = repeat(ys, train_years)
ys_test = repeat(ys, test_years);

In [ ]:
# Normalize the data, divide by std not possible due to zeros

X_train_norm = (train_data .- mean(train_data, dims=3)) #./ std(train_data, dims=3)
X_test_norm = (test_data .- mean(test_data, dims=3)) #./ std(test_data, dims = 3);

#add channel dimension
# flux expects data to be [width, height, channels, samples]

X_train = reshape(X_train_norm, (size(X_train_norm, 1),  size(X_train_norm, 2), 1, size(X_train_norm, 3)))
X_test = reshape(X_test_norm, (size(X_test_norm, 1),  size(X_test_norm, 2), 1, size(X_test_norm, 3)));

In [ ]:
ys_train = Flux.onehotbatch(ys_train, 1:12)
ys_test = Flux.onehotbatch(ys_test, 1:12);

In [ ]:
# Define the CNN model
model = Chain(
    Conv((5, 5), 1=>4, relu),
    MaxPool((2, 2)),
    Conv((3, 3), 4=>8, relu),
    MaxPool((2, 2)),
    Conv((3, 3), 8=>16, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(224, 64, relu),
    Dense(64, num_months),
    softmax
)

In [ ]:
train_loader = Flux.DataLoader((X_train, ys_train), batchsize=32, shuffle=true)
test_loader = Flux.DataLoader((X_test, ys_test), batchsize=32)

## Check accuracy before training:

In [ ]:
function accuracy(loader)
    acc = 0.0
    for (X_batch, y_batch) in loader
        acc += mean(Flux.onecold(model(X_batch)) .== Flux.onecold(y_batch))
    end
    return acc / length(loader)
end

In [ ]:
println("Training accuracy: ", accuracy(train_loader))
println("Testing accuracy: ", accuracy(test_loader))

## Training

In [ ]:
# Define the loss function and optimizer
loss(model, x, y) = Flux.crossentropy(model(x), y) # model is first argument
opt = Flux.setup(Adam(), model)

# Train the model
epochs = 10

for epoch in 1:epochs
    Flux.train!(loss, model, train_loader, opt)
    println("Epoch $epoch complete")
end

In [ ]:
println("Training accuracy: ", accuracy(train_loader))
println("Testing accuracy: ", accuracy(test_loader))

In [ ]:
# Save the model
using BSON: @save
@save "mymodel.bson" model

# Gaussian Processes

Use the data above to create a toy GP problem. E.g. you could predict SST at a certain gridpoint from neighbouring gridpoints.  Research https://github.com/JuliaGaussianProcesses for more.